# Import Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pickle
import yagmail
from tensorflow.keras.models import load_model

# Loading Models

$$ ArtificialNeuralNetwork $$

In [2]:
model=load_model(filepath="models/ArtificialNeuralNetwork_model.h5")

$$ Loading Scaler model $$

In [3]:
with open(file="models/scaler.pkl",mode="rb") as file:
    scaler=pickle.load(file=file)

In [4]:
class_labels=['normal','dos','r2l','probe','u2r']

In [5]:
IPs_file=pd.read_excel("testing_data/Blocked_IP_Addresses.xlsx")
IPs_file=IPs_file.head(5)
IPs_file.to_excel("testing_data/Blocked_IP_Addresses.xlsx",index=False)
IPs_file=pd.read_excel("testing_data/Blocked_IP_Addresses.xlsx")
IPs_file.head(10)

,Attack_Ips
0,192.168.1.193
1,192.168.1.152
2,192.168.1.169
3,192.168.1.79
4,192.168.1.33


In [6]:
hash_file=pd.read_excel("testing_data/Blocked_HashCodes.xlsx")
hash_file=hash_file.head(3).astype(str)
hash_file.to_excel("testing_data/Blocked_HashCodes.xlsx",index=False)
hash_file=pd.read_excel("testing_data/Blocked_HashCodes.xlsx")
hash_file.head()

,Intrusion_HashCodes
0,10651105497456949248
1,4109532635900570112
2,8736532635932669952


# User Input

In [7]:
input_=pd.read_excel("testing_data/file_1.xlsx")
input_.head()

,IP_Add,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,172.17.0.5,0,0,0.028986,0,0,0,0,0,0,...,1,0,0,1,0,0,0.06,0,0.92,1


---

In [8]:
ip_address=input_.pop("IP_Add")
inp_cols=input_.columns
scaled_input_=scaler.transform(input_)
input_df=pd.DataFrame(data=scaled_input_,columns=inp_cols)
input_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0,0.0,-0.014073,-0.1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.003937,0.0,1.0,0.0,0.0,0.06,0.0,0.92,1.0


In [9]:
model_pred=model.predict(input_df)
model_pred_class=np.argmax(model_pred[0])
model_pred_label=class_labels[model_pred_class]
if model_pred_label!="normal":
    new_ip_data=pd.DataFrame()
    new_hashcode_data=pd.DataFrame()
    new_ip_data['Attack_Ips']=ip_address
    new_hashcode_data['Intrusion_HashCodes']=[str(pd.util.hash_pandas_object(input_.iloc[[0]])[0])]
    final_ip_file=pd.concat(objs=[IPs_file,new_ip_data],ignore_index=True)
    final_hash_file=pd.concat(objs=[hash_file,new_hashcode_data],ignore_index=True)
    final_ip_file.to_excel("testing_data/Blocked_IP_Addresses.xlsx",index=False)
    final_hash_file.to_excel("testing_data/Blocked_HashCodes.xlsx",index=False)
    
    user = yagmail.SMTP(user='mohanakt6@gmail.com',password='epzivrkcskkzdtym')
    user.send(to='mohanakt6@gmail.com',subject='$$ALERT$$',contents=f"$$ {model_pred_label}  intrusion $$ has been found.")
    print("Email sent sucessfully")
IPs_file=pd.read_excel("testing_data/Blocked_IP_Addresses.xlsx")
hash_file=pd.read_excel("testing_data/Blocked_HashCodes.xlsx",dtype=str)
display("ArtificialNeuralNetwork model predicted class is:",HTML(str(model_pred_class)))
display("ArtificialNeuralNetwork model predicted label is:",HTML(str(model_pred_label)))

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials z1-20020a1709028f8100b001888cadf8f6sm3793266plo.49 - gsmtp')

In [ ]:
IPs_file.head(100)

In [ ]:
hash_file.head(100)

---

---

---